### Pipeline that Generates key based on Local Time and Fibonacci Sequences

You can find the full README at:

https://github.com/ydataai/academy/blob/master/tutorials/Pipelines/README.md


### Input variables


In [ ]:
# The python packages that will be installed in the multiple pipeline steps
pip_packages_to_install = [
    "datetime"
]

# Kfp
pipeline_name = "Generates key based on Local Time and Fibonacci Sequences"
pipeline_description = "Generates key based on Local Time and Fibonacci Sequences"
experiment_name = "Fibonacci experiment"
run_name = "Sample Run"


In [ ]:
import kfp
import typing


#### Token Generator

In [ ]:
from typing import NamedTuple
from kfp.components import *

def token_generator() -> str:
    from datetime import datetime

    now = datetime.now()
    token = now.strftime("%H%M")
    while len(token) < 4:
        token = str(0) + str(token)
    return token


In [ ]:
kfp_token_generator = kfp.components.func_to_container_op(
    func=token_generator,
    output_component_file="./tok_gen.yaml",
    packages_to_install=pip_packages_to_install,
)


#### Unique Key Generated based on the Token Passed

In [ ]:
def key_gen(tok: str) -> NamedTuple("S_outs", [("outkey", int), ("token", str)]):

    tok = str(tok)

    def fib_generate(a, s):
        p = 0
        q = s
        r = s
        for x in range(a - 1):
            r = p + q
            p = q
            q = r
        return r

    out_key = fib_generate(int(str(tok[1]) + str(tok[3])), int(tok[0]) + 1)
    print("Generated key {}".format(out_key))
    return (out_key, tok)


#### Key Decoder - Decoded key matches with Passed key only if Token is Same

In [ ]:
def key_decode(tok: str, seq_two: int) -> str:
    def fib_generate(a, s):
        p = 0
        q = s
        r = s
        for x in range(a - 1):
            r = p + q
            p = q
            q = r
        return r

    def fib_decode(f, s):
        x = 0
        y = fib_generate(0, s)
        while f >= fib_generate(x, s):
            y = fib_generate(x, s)
            x += 1
        return y

    if fib_decode(seq_two, int(tok[0]) + 1) == seq_two:
        print("Keys match")
        out_str = "Keys Match"
    else:
        print("Keys don't match")
        out_str = "Keys Don't Match"

    return out_str


In [ ]:
kfp_key_gen = kfp.components.func_to_container_op(
    func=key_gen, output_component_file="./kgen.yaml"
)


In [ ]:
kfp_key_decode = kfp.components.func_to_container_op(
    func=key_decode, output_component_file="./kdecode.yaml"
)


#### Token Generator 2

In [ ]:
def token_generator_two() -> str:
    from datetime import datetime

    now = datetime.now()
    token = now.strftime("%H%M")
    token = int(token) + 7
    while len(str(token)) < 4:
        token = str(0) + str(token)
    return str(token)


In [ ]:
kfp_token_generator_two = kfp.components.func_to_container_op(
    func=token_generator_two,
    output_component_file="./tok_gen_two.yaml",
    packages_to_install=pip_packages_to_install,
)


#### Defining the Pipeline Execution Sequence and Input-Output scheme

In [ ]:
import kfp.dsl as dsl

@dsl.pipeline(
    name="Fibonacci Security Key Generator",
    description="Generates automatically a random key from a token based on Fibonacci sequences",
)
def Keygen_func():
    # Passing pipeline parameter and a constant value as operation arguments
    tg_one_task = kfp_token_generator()
    kg_one_task = kfp_key_gen(tg_one_task.output)
    kg_two_task = kfp_key_gen(tg_one_task.output)
    tg_two_task = kfp_token_generator_two()
    kg_three_task = kfp_key_gen(tg_two_task.output)
    kd_one_task = kfp_key_decode(
        kg_one_task.outputs["token"], kg_two_task.outputs["outkey"]
    )
    kd_two_task = kfp_key_decode(
        kg_two_task.outputs["token"], kg_three_task.outputs["outkey"]
    )
    kd_three_task = kfp_key_decode(
        kg_three_task.outputs["token"], kg_two_task.outputs["outkey"]
    )


# For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
# For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax


#### Compiling the Pipeline

In [ ]:
pipeline_func = Keygen_func
pipeline_filename = pipeline_func.__name__ + ".pipeline.tar.gz"

import kfp.compiler as comp

comp.Compiler().compile(pipeline_func, pipeline_filename)

#### Running the Pipeline

In [ ]:
import json
import uuid
from random import randrange

client = kfp.Client()

# check if pipeline already exists -> if not, create a new one
filter = json.dumps(
    {
        "predicates": [
            {"key": "name", "op": 1, "string_value": "{}".format(pipeline_name)}
        ]
    }
)
pipeline = client.pipelines.list_pipelines(filter=filter)

if pipeline.pipelines is None:
    print("Creating a new pipeline: " + pipeline_name)
    pipeline = client.pipeline_uploads.upload_pipeline(
        pipeline_filename, name=pipeline_name, description=pipeline_description
    )
else:
    print("Pipeline already exists: " + pipeline_name)
    pipeline = pipeline.pipelines[0]
    
pipeline_version = str(uuid.uuid4())

print("Creating a new pipeline version: " + pipeline_name + str(" [" + pipeline_version + "]"))
client.pipeline_uploads.upload_pipeline_version(
    pipeline_filename,
    name=pipeline_name + str(" [" + pipeline_version + "]"),
    pipelineid=pipeline.id,
)

# create a new experiment or use an existing one
print("Creating a new experiment or use an existing one: " + experiment_name)
experiment = client.create_experiment(name=experiment_name)

# create a new run with a random identifier
run_random_id = str(randrange(1000))
print("Creating a new run: " + run_name + " [" + run_random_id + "]")
new_run = client.run_pipeline(
    experiment.id, run_name + " [" + run_random_id + "]", pipeline_id=pipeline.id
)
